In [1]:
import pandas as pd
import os
import json
import glob
import http.client
from pandasql import sqldf
from re import sub
from unidecode import unidecode

### Opcionais caso necessite instalar

In [ ]:
pip install Unidecode

In [ ]:
pip install pandasql

# Carregando Arquivos

In [2]:
def get_files(filepath, filetype="json"):
    all_files = []
    dfs = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root, '*.'+filetype)) # list all files the folder
        for f in files:
            all_files.append(os.path.abspath(f))
    for file in all_files:
        if filetype == "json":
            data = pd.read_json(file, lines=False, encoding='utf-8-sig') # read data frame from json file
        elif filetype == "csv":
            data = pd.read_csv(file, sep=None, engine='python', encoding='utf-8-sig') # read data frame from csv file
        dfs.append(data)
    return pd.concat(dfs, ignore_index=True)


dfVRA = get_files("data_engineer_test/VRA")
dfAIR_CIA = get_files("data_engineer_test/AIR_CIA","csv")

In [3]:
print(len(dfVRA))
dfVRA.head()

535803


,ICAOEmpresaAérea,NúmeroVoo,CódigoAutorização,CódigoTipoLinha,ICAOAeródromoOrigem,ICAOAeródromoDestino,PartidaPrevista,PartidaReal,ChegadaPrevista,ChegadaReal,SituaçãoVoo,CódigoJustificativa
0,AZU,4478,0,N,SBRJ,SBGO,2021-06-30 22:15:00,2021-06-30 22:00:00,2021-06-30 23:55:00,2021-06-30 23:56:00,REALIZADO,N/A
1,AZU,4479,0,N,SBGO,SBRJ,2021-06-07 06:20:00,2021-06-07 06:14:00,2021-06-07 08:00:00,2021-06-07 07:47:00,REALIZADO,N/A
2,AZU,4479,0,N,SBGO,SBRJ,2021-06-08 06:20:00,2021-06-08 06:13:00,2021-06-08 08:00:00,2021-06-08 07:48:00,REALIZADO,N/A
3,AZU,4479,0,N,SBGO,SBRJ,2021-06-09 06:20:00,2021-06-09 06:10:00,2021-06-09 08:00:00,2021-06-09 07:55:00,REALIZADO,N/A
4,AZU,4479,0,N,SBGO,SBRJ,2021-06-10 06:20:00,2021-06-10 06:13:00,2021-06-10 08:00:00,2021-06-10 07:53:00,REALIZADO,N/A


In [4]:
print(len(dfAIR_CIA))
dfAIR_CIA.head()

20


,Razão Social,ICAO IATA,CNPJ,Atividades Aéreas,Endereço Sede,Telefone,E-Mail,Decisão Operacional,Data Decisão Operacional,Validade Operacional
0,ABSA - AEROLINHAS BRASILEIRAS S.A.,LTG M3,00.074.635/0001-33,TRANSPORTE AÉREO REGULAR,"AEROPORTO INTERNACIONAL DE VIRACOPOS, RODOVIA ...",(11) 5582-8055,gar@tam.com.br,DECISÃO Nº 41,22/04/2015,23/04/2025
1,AEROSUL TÁXI AÉREO LTDA (EX.: AUSTEN TÁXI AÉRE...,ASO 2S,27.315.694/0001-02,"SERVIÇOS AÉREOS PÚBLICOS, TÁXI-AÉREO, TRANSPOR...","RODOVIA PR 218, KM 7, JARDIM UNIVERSITÁRIO, AE...",(43) 3176-4030,operacoes@aerosul.com.br,DECISÃO Nº 282,10/02/2021,NaN
2,ASTA LINHAS AÉREAS LTDA ( EX - AMÉRICA DO SUL ...,SUL 0A,12.703.737/0001-50,"SERVIÇOS AÉREOS PÚBLICOS, TRANSPORTE AÉREO REG...","AV GOVERNADOR JOÃO PONCE DE ARRUDA, S/N, HANGA...",(65) 3614-2684 / (18) 2104-1000,NaN,DECISÃO Nº 115,31/07/2019,02/08/2024
3,ATA - AEROTÁXI ABAETÉ LTDA.,ABJ,14.674.451/0001-19,"TÁXI-AÉREO, TRANSPORTE AÉREO NÃO REGULAR, TRAN...","PÇA. GAGO COUTINHO, S/N, AEROPORTO INTERNACIO...",(71) 34629600 / (71) 996643563,guilherme@abaete.com.br,DECISÃO Nº 36,05/03/2020,NaN
4,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),ACN 2F,04.263.318/0001-16,TRANSPORTE AÉREO REGULAR,"AVENIDA EMILIO ANTONON, 901, BAIRRO CHÁCARA AE...",(11) 45822355/ (11) 987433988,NaN,"Decisão nº 14, de 25/01/2018",25/01/2018,29/01/2028


# Colunas para snake_case

In [5]:
print(dfVRA.columns)
print(dfAIR_CIA.columns)

Index(['ICAOEmpresaAérea', 'NúmeroVoo', 'CódigoAutorização', 'CódigoTipoLinha',
       'ICAOAeródromoOrigem', 'ICAOAeródromoDestino', 'PartidaPrevista',
       'PartidaReal', 'ChegadaPrevista', 'ChegadaReal', 'SituaçãoVoo',
       'CódigoJustificativa'],
      dtype='object')
Index(['Razão Social', 'ICAO IATA', 'CNPJ', 'Atividades Aéreas',
       'Endereço Sede', 'Telefone', 'E-Mail', 'Decisão Operacional',
       'Data Decisão Operacional', 'Validade Operacional'],
      dtype='object')


In [6]:
def snake_case(s):
  return '_'.join(
    sub(r"(\s|_|-)+"," ",
    sub(r"[A-Z]{2,}(?=[A-Z][a-z]+[0-9]*|\b)|[A-Z]?[a-z]+[0-9]*|[A-Z]|[0-9]+",
    lambda mo: ' ' + mo.group(0).lower(), s)).split())

In [7]:
dfVRA = dfVRA.rename(mapper=unidecode, axis=1)

In [8]:
dfVRA = dfVRA.rename(mapper=snake_case, axis=1)

In [9]:
dfVRA.columns

Index(['icao_empresa_aerea', 'numero_voo', 'codigo_autorizacao',
       'codigo_tipo_linha', 'icao_aerodromo_origem', 'icao_aerodromo_destino',
       'partida_prevista', 'partida_real', 'chegada_prevista', 'chegada_real',
       'situacao_voo', 'codigo_justificativa'],
      dtype='object')

In [10]:
dfAIR_CIA = dfAIR_CIA.rename(mapper=unidecode, axis=1)

In [11]:
dfAIR_CIA = dfAIR_CIA.rename(mapper=snake_case, axis=1)

In [12]:
dfAIR_CIA.columns

Index(['razao_social', 'icao_iata', 'cnpj', 'atividades_aereas',
       'endereco_sede', 'telefone', 'e_mail', 'decisao_operacional',
       'data_decisao_operacional', 'validade_operacional'],
      dtype='object')

# Separando a coluna ICAO IATA

In [13]:
dfAIR_CIA[['icao','iata']]=dfAIR_CIA.icao_iata.str.split(expand=True)

In [14]:
dfAIR_CIA.head()

,razao_social,icao_iata,cnpj,atividades_aereas,endereco_sede,telefone,e_mail,decisao_operacional,data_decisao_operacional,validade_operacional,icao,iata
0,ABSA - AEROLINHAS BRASILEIRAS S.A.,LTG M3,00.074.635/0001-33,TRANSPORTE AÉREO REGULAR,"AEROPORTO INTERNACIONAL DE VIRACOPOS, RODOVIA ...",(11) 5582-8055,gar@tam.com.br,DECISÃO Nº 41,22/04/2015,23/04/2025,LTG,M3
1,AEROSUL TÁXI AÉREO LTDA (EX.: AUSTEN TÁXI AÉRE...,ASO 2S,27.315.694/0001-02,"SERVIÇOS AÉREOS PÚBLICOS, TÁXI-AÉREO, TRANSPOR...","RODOVIA PR 218, KM 7, JARDIM UNIVERSITÁRIO, AE...",(43) 3176-4030,operacoes@aerosul.com.br,DECISÃO Nº 282,10/02/2021,NaN,ASO,2S
2,ASTA LINHAS AÉREAS LTDA ( EX - AMÉRICA DO SUL ...,SUL 0A,12.703.737/0001-50,"SERVIÇOS AÉREOS PÚBLICOS, TRANSPORTE AÉREO REG...","AV GOVERNADOR JOÃO PONCE DE ARRUDA, S/N, HANGA...",(65) 3614-2684 / (18) 2104-1000,NaN,DECISÃO Nº 115,31/07/2019,02/08/2024,SUL,0A
3,ATA - AEROTÁXI ABAETÉ LTDA.,ABJ,14.674.451/0001-19,"TÁXI-AÉREO, TRANSPORTE AÉREO NÃO REGULAR, TRAN...","PÇA. GAGO COUTINHO, S/N, AEROPORTO INTERNACIO...",(71) 34629600 / (71) 996643563,guilherme@abaete.com.br,DECISÃO Nº 36,05/03/2020,NaN,ABJ,None
4,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),ACN 2F,04.263.318/0001-16,TRANSPORTE AÉREO REGULAR,"AVENIDA EMILIO ANTONON, 901, BAIRRO CHÁCARA AE...",(11) 45822355/ (11) 987433988,NaN,"Decisão nº 14, de 25/01/2018",25/01/2018,29/01/2028,ACN,2F


# Salvando arquivos

In [15]:
dfAIR_CIA.to_csv('dfAIR_CIA_out.csv', index=True)
dfVRA.to_json('dfVRA.json', orient="records", lines=False)

# Carregando dados Rapid API

In [16]:
dfICAO = dfVRA['icao_aerodromo_origem']
ICAO_codes = dfICAO.unique()
ICAO_codes

array(['SBRJ', 'SBGO', 'SBBR', 'SBRF', 'SBFL', 'SBKP', 'SBPV', 'SBJP',
       'SBEG', 'SBPA', 'SBSG', 'SBGR', 'SBCH', 'SBCF', 'SBVT', 'SBCY',
       'SBMK', 'SBSI', 'SBJU', 'SBPL', 'SBBE', 'SBAR', 'SBDN', 'SBMO',
       'SBAE', 'SBAT', 'SBPS', 'SBGL', 'SBJE', 'SBSV', 'SBNF', 'SBTE',
       'SBCT', 'SBLO', 'SBCA', 'SBJV', 'SBUL', 'SBKG', 'SBIL', 'SBCP',
       'SBSP', 'SBFZ', 'SBME', 'SBZM', 'SBGV', 'SBMS', 'SBPJ', 'SBSM',
       'SBNM', 'SBUG', 'SBCG', 'SBJA', 'SBMG', 'SBCX', 'SBSR', 'SBAU',
       'SBVH', 'SBFI', 'SBRP', 'SBSL', 'KFLL', 'LPPT', 'SBPK', 'SGAS',
       'SEGU', 'SMJP', 'SUMU', 'MTPP', 'SBRB', 'MGGT', 'EBBR', 'SBIH',
       'SBUA', 'SWCA', 'SBTF', 'SWEI', 'SWKO', 'SWLB', 'SWPI', 'KPUC',
       'SBUY', 'SBTB', 'KMIA', 'SKBO', 'SGES', 'SPJC', 'FNLU', 'LEMD',
       'KIAH', 'KEWR', 'KORD', 'LFPG', 'SAEZ', 'SABE', 'EDDF', 'PANC',
       'RKSI', 'KLAX', 'SCEL', 'SEQM', 'EHAM', 'GVAC', 'MMMX', 'LIMC',
       'SBMA', 'LIRF', 'SBMQ', 'SBBV', 'SBIZ', 'SBSN', 'KJFK', 'MMUN',
      

In [17]:
def get_airports(codes):
    conn = http.client.HTTPSConnection("airport-info.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': "ce19d0164fmsh3d383efc0e85ce5p16dcb1jsnb1a4a3c79541",
        'x-rapidapi-host': "airport-info.p.rapidapi.com"
        }
    res = []
    for code in codes:
        conn.request("GET", f"/airport?icao={code}", headers=headers)
        resp =  conn.getresponse()
        data = resp.read().decode("utf-8-sig")
        res.append(data)
    return res

In [18]:
airports = get_airports(ICAO_codes)

In [19]:
len(airports)

301

In [20]:
def airports_to_df(airports):
    dfs = []
    for i in range(len(airports)):
        df = json.loads(airports[i])
        df = pd.DataFrame(df, index=[0])
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

In [21]:
dfAir = airports_to_df(airports)

In [22]:
dfAir = dfAir.drop(columns=['error'])

In [23]:
dfAir["county_state"] = dfAir["county"] + "-" + dfAir["state"]

In [24]:
dfAir.head()

,id,iata,icao,name,location,street_number,street,city,county,state,country_iso,country,postal_code,phone,latitude,longitude,uct,website,county_state
0,6679.0,SDU,SBRJ,Santos Dumont Airport,"Rio de Janeiro, Rio de Janeiro, Brazil",s/n,Praça Senador Salgado Filho,,Rio de Janeiro,Rio de Janeiro,BR,Brazil,20021-340,+55 21 3814-7070,-22.911144,-43.164875,-180.0,http://www4.infraero.gov.br/aeroportos/aeropor...,Rio de Janeiro-Rio de Janeiro
1,2787.0,GYN,SBGO,Santa Genoveva Airport,"Goiânia, Goiás, Brazil",s/n,Alameda 4,,Goiânia,Goiás,BR,Brazil,74672-410,+55 62 3265-1500,-16.632303,-49.216347,-180.0,http://www4.infraero.gov.br/aeroportos/aeropor...,Goiânia-Goiás
2,953.0,BSB,SBBR,Brasília International Airport (Presidente J. ...,"Brasília, Distrito Federal, Brazil",,Lago Sul,,Brasília,Distrito Federal,BR,Brazil,71608-900,+55 61 3364-9000,-15.869737,-47.917236,-180.0,http://www.bsb.aero/,Brasília-Distrito Federal
3,6335.0,REC,SBRF,Recife/Guararapes–Gilberto Freyre Internationa...,"Recife, Pernambuco, Brazil",s/n,Praça Ministro Salgado Filho,,Recife,Pernambuco,BR,Brazil,51210-902,+55 81 3322-4188,-8.125932,-34.924015,-180.0,http://www4.infraero.gov.br/aeroportos/aeropor...,Recife-Pernambuco
4,2320.0,FLN,SBFL,Hercílio Luz International Airport,"Florianópolis, Santa Catarina, Brazil",3393,Avenida Deputado Diomício Freitas,,Florianópolis,Santa Catarina,BR,Brazil,88047-900,+55 48 3331-4000,-27.670118,-48.545967,-180.0,http://www.infraero.gov.br/index.php/br/aeropo...,Florianópolis-Santa Catarina


In [25]:
dfAir.to_json('dfAir_out.json', orient="records", lines=False)

# Criando Views com PandasSQL

In [26]:
pysqldf = lambda q: sqldf(q, globals())

###### Para cada companhia aérea trazer a rota mais utilizada com as seguintes informações:
 - Razão social da companhia aérea
 - Nome Aeroporto de Origem
 - ICAO do aeroporto de origem
 - Estado/UF do aeroporto de origem
 - Nome do Aeroporto de Destino
 - ICAO do Aeroporto de destino
 - Estado/UF do aeroporto de destino

In [27]:
query_top_route = '''
            WITH rotas AS (SELECT *, max(qt_rotas) FROM (SELECT icao_empresa_aerea as icao_cia, numero_voo as rota, 
                count(numero_voo) as qt_rotas, icao_aerodromo_origem as aero_og, icao_aerodromo_destino as aero_dest 
                FROM dfVRA
                WHERE situacao_voo = "REALIZADO"
                GROUP BY rota
                ORDER BY qt_rotas DESC) GROUP BY icao_cia ORDER BY max(qt_rotas) DESC),
            origem as (SELECT icao_aerodromo_origem, dfAir.name as aero_origem, 
                dfAir.county_state as uf_estado_origem
                FROM dfVRA INNER JOIN dfAir
                ON dfVRA.icao_aerodromo_origem = dfAir.icao
                GROUP BY icao_aerodromo_origem),
            destino as (SELECT icao_aerodromo_destino, dfAir.name as aero_destino, 
                dfAir.county_state as uf_estado_destino
                FROM dfVRA INNER JOIN dfAir
                ON dfVRA.icao_aerodromo_destino = dfAir.icao
                GROUP BY icao_aerodromo_destino),
            cias as (SELECT razao_social, icao FROM dfAIR_CIA
                GROUP BY icao)
            
            SELECT cias.razao_social, origem.aero_origem, rotas.aero_og, origem.uf_estado_origem,
            destino.aero_destino, rotas.aero_dest, destino.uf_estado_destino FROM rotas 
            INNER JOIN origem ON
                rotas.aero_og = origem.icao_aerodromo_origem
            INNER JOIN destino ON
                rotas.aero_dest = destino.icao_aerodromo_destino
            INNER JOIN cias ON
                rotas.icao_cia = cias.icao
                
            '''

pysqldf(query_top_route)            

,razao_social,aero_origem,aero_og,uf_estado_origem,aero_destino,aero_dest,uf_estado_destino
0,AZUL LINHAS AÉREAS BRASILEIRAS S/A,Santarém-Maestro Wilson Fonseca Airport,SBSN,Santarém-Pará,Eduardo Gomes International Airport,SBEG,Manaus-Amazonas
1,SIDERAL LINHAS AÉREAS LTDA.,Tancredo Neves International Airport (Confins ...,SBCF,Confins-Minas Gerais,Deputado Luís Eduardo Magalhães International ...,SBSV,Salvador-Bahia
2,PASSAREDO TRANSPORTES AÉREOS S.A.,Santa Genoveva Airport,SBGO,Goiânia-Goiás,Leite Lopes Airport,SBRP,Ribeirão Preto-São Paulo
3,TAM LINHAS AÉREAS S.A.,Santos Dumont Airport,SBRJ,Rio de Janeiro-Rio de Janeiro,São Paulo–Guarulhos International Airport,SBGR,Guarulhos-São Paulo
4,GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS ...,Recife/Guararapes–Gilberto Freyre Internationa...,SBRF,Recife-Pernambuco,São Paulo–Guarulhos International Airport,SBGR,Guarulhos-São Paulo
5,TOTAL LINHAS AÉREAS S.A.,São Paulo–Guarulhos International Airport,SBGR,Guarulhos-São Paulo,Rio de Janeiro–Galeão International Airport,SBGL,Rio de Janeiro-Rio de Janeiro
6,MAP TRANSPORTES AÉREOS LTDA.,Itaituba Airport,SBIH,Itaituba-State of Pará,Eduardo Gomes International Airport,SBEG,Manaus-Amazonas
7,MODERN TRANSPORTE AEREO DE CARGA S.A,Eduardo Gomes International Airport,SBEG,Manaus-Amazonas,Brasília International Airport (Presidente J. ...,SBBR,Brasília-Distrito Federal
8,ABSA - AEROLINHAS BRASILEIRAS S.A.,Viracopos/Campinas International Airport,SBKP,Campinas-São Paulo,Abu Dhabi International Airport,OMAA,-Abu Dhabi
9,AEROSUL TÁXI AÉREO LTDA (EX.: AUSTEN TÁXI AÉRE...,Afonso Pena International Airport,SBCT,São José dos Pinhais-Paraná,Caçador Airport,SBCD,Caçador-Santa Catarina


###### Para cada aeroporto trazer a companhia aérea com maior atuação no ano com as seguintes informações:
 - Nome do Aeroporto
 - ICAO do Aeroporto
 - Razão social da Companhia Aérea
 - Quantidade de Rotas à partir daquele aeroporto
 - Quantidade de Rotas com destino àquele aeroporto
 - Quantidade total de pousos e decolagens naquele aeroporto

In [28]:
query_top_cia = '''WITH aeros as (SELECT icao, name FROM dfAir),
                top_from as (SELECT *, max(nr_rotas) as rotas_origem FROM (SELECT icao_empresa_aerea, 
                    COUNT(DISTINCT numero_voo) as nr_rotas, icao_aerodromo_origem
                    FROM dfVRA WHERE situacao_voo = 'REALIZADO'
                    GROUP BY icao_aerodromo_origem, icao_empresa_aerea
                    ORDER BY nr_rotas DESC) GROUP BY icao_aerodromo_origem ORDER BY nr_rotas DESC),
                top_to as (SELECT *, max(nr_rotas) as rotas_destino FROM (SELECT icao_empresa_aerea, 
                    COUNT(DISTINCT numero_voo) as nr_rotas, 
                    icao_aerodromo_destino
                    FROM dfVRA WHERE situacao_voo = 'REALIZADO'
                    GROUP BY icao_aerodromo_destino, icao_empresa_aerea
                    ORDER BY nr_rotas DESC) 
                    GROUP BY icao_aerodromo_destino ORDER BY nr_rotas DESC),
                top_departures as (SELECT *, max(departures) as total_departures FROM (SELECT icao_empresa_aerea, 
                    icao_aerodromo_origem, count(icao_aerodromo_origem) as departures
                    FROM dfVRA
                    GROUP BY icao_aerodromo_origem, icao_empresa_aerea
                    ORDER BY departures DESC) GROUP BY icao_aerodromo_origem),
                top_arrivals as (SELECT *, max(arrivals) as total_arrivals FROM (SELECT icao_empresa_aerea, 
                    icao_aerodromo_destino, count(icao_aerodromo_destino) as arrivals
                    FROM dfVRA
                    GROUP BY icao_aerodromo_destino, icao_empresa_aerea
                    ORDER BY arrivals DESC) GROUP BY icao_aerodromo_destino),
                cias as (SELECT razao_social, icao FROM dfAIR_CIA
                    GROUP BY icao)


SELECT aeros.name as airport, aeros.icao, cias.razao_social as cia, top_from.rotas_origem as routes_from, 
top_to.rotas_destino as routes_to, top_departures.total_departures, top_arrivals.total_arrivals
FROM aeros INNER JOIN
top_from ON
aeros.icao = top_from.icao_aerodromo_origem
INNER JOIN cias ON
top_from.icao_empresa_aerea = cias.icao
INNER JOIN top_to ON
aeros.icao = top_to.icao_aerodromo_destino
INNER JOIN top_departures ON
aeros.icao = top_departures.icao_aerodromo_origem
INNER JOIN top_arrivals ON
aeros.icao = top_arrivals.icao_aerodromo_destino
limit 50
'''
pysqldf(query_top_cia)

,airport,icao,cia,routes_from,routes_to,total_departures,total_arrivals
0,São Paulo–Guarulhos International Airport,SBGR,TAM LINHAS AÉREAS S.A.,516,530,33173,33181
1,Viracopos/Campinas International Airport,SBKP,AZUL LINHAS AÉREAS BRASILEIRAS S/A,438,468,41761,41662
2,São Paulo–Congonhas Airport,SBSP,TAM LINHAS AÉREAS S.A.,339,346,15026,15015
3,Brasília International Airport (Presidente J. ...,SBBR,TAM LINHAS AÉREAS S.A.,264,273,16174,16170
4,Tancredo Neves International Airport (Confins ...,SBCF,AZUL LINHAS AÉREAS BRASILEIRAS S/A,263,255,20987,20952
5,Recife/Guararapes–Gilberto Freyre Internationa...,SBRF,AZUL LINHAS AÉREAS BRASILEIRAS S/A,196,200,20492,20499
6,Rio de Janeiro–Galeão International Airport,SBGL,GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS ...,177,181,6302,6437
7,Santos Dumont Airport,SBRJ,AZUL LINHAS AÉREAS BRASILEIRAS S/A,163,175,11990,11939
8,Salgado Filho International Airport,SBPA,AZUL LINHAS AÉREAS BRASILEIRAS S/A,139,123,7914,7931
9,Deputado Luís Eduardo Magalhães International ...,SBSV,GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS ...,122,130,6736,6596


# Extras:
### Descrever qual estratégia você usaria para ingerir estes dados de forma incremental caso precise capturar esses dados a cada mes?
 - Utilizando o Apache Airflow realizaria a extração dos dados RAW de VRA, AIR_CIA e Airports caso estes dados estejam em bancos de terceiros e carregaria estes dados na TRANSIENT_ZONE do Data Lake com permanência de 30 dias para backup mais recente. Na sequência esses dados seriam transferidos para a RAW_ZONE com aplicação de metadados e onde seriam mantidos para histórico. 
 - Em seguida, estes dados seriam extraídos da RAW_ZONE e transformados via Spark através de Cluster do GCP Dataproc ou AWS EMR para limpeza das colunas e qualidade dos dados e após seriam carregados na TRUSTED_ZONE como dados validados. 
 - Para a última etapa, utilizando novamente o Spark, os dados seriam transformados para gerar o enriquecimento dos dados e criação das views necessárias para os departamentos de Produto e BI. Esses dados finais seriam salvos na REFINED_ZONE do Data Lake e com uma cópia nos sistemas de consumo (GCP BigQuery ou AWS Redshift), que podem ser acessados via API.

### Justifique em cada etapa sobre a escalabilidade da tecnologia utilizada.
 - Apache Airflow devido a escalabilidade de configuração dos workers para a demanda do serviço além das muitas bibliotecas para operadores que facilitam integrações com serviços e sistemas de terceiros.
 - Apache Spark devido ao processamento em paralelo e facilidade de implementação de Jobs via PySpark, além de que é possível lançar jobs na infraestrutra da GCP ou AWS com fácil configuracão dos clusters de máquinas e pricing proporcional ao volume da aplicação.  

### Justifique as camadas utilizadas durante o processo de ingestão até a disponibilização dos dados.
 - Captura dos Dados 30 dias: TRANSIENT_ZONE
 - Manutencão Histórico dos Dados: RAW_ZONE
 - DataFrames tratados e carregados: TRUSTED_ZONE
 - Views SQL: REFINED_ZONE

# FIM